In [11]:
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=2.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

def process_frame(img):
    
    if cv2.waitKey(50) == 32:
        cv2.imwrite('snapshot.png', img)
        print("Snapshot created.")
    
    img_final = np.array(img, copy=True)
    img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Gray Image", img2gray)
       
    ret, mask = cv2.threshold(img2gray, 180, 255, cv2.THRESH_BINARY)
    cv2.imshow('Treshold mask', mask)
    
    image_final = cv2.bitwise_and(img2gray, img2gray, mask=mask)
    cv2.imshow('Image final', image_final)
    
    image_final_sharp = unsharp_mask(image_final)
    cv2.imshow("Sharpened final", image_final_sharp)
    
    ret, new_img = cv2.threshold(image_final, 180, 255, cv2.THRESH_BINARY_INV)  # for black text , cv.THRESH_BINARY_INV
    cv2.imshow("New_img", new_img)
    '''
            line  8 to 12  : Remove noisy portion 
    '''
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,
                                                         3))# to manipulate the orientation of dilution , large x means horizonatally dilating  more, large y means vertically dilating more
    
    dilated = cv2.dilate(new_img, kernel, iterations=7)  # dilate , more the iteration more the dilation
    
    # for cv2.x.x

    # _, contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # findContours returns 3 variables for getting contours

    # for cv3.x.x comment above line and uncomment line below

    image, contours, hierarchy = cv2.findContours(dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    index = 0;
    textStack = []

    for contour in contours:
        # get rectangle bounding contour
        [x, y, w, h] = cv2.boundingRect(contour)
        
        cv2.rectangle(dilated, (x, y), (x + w, y + h), (255, 0, 255), 2)

        # Don't plot small false positives that aren't text
        if w < 35 and h < 35:
            continue

        # draw rectangle around contour on original image
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 2)

        
        #you can crop image and send to OCR  , false detected will return no text :)
        cropped = image_final[y :y +  h , x : x + w]
        
        textStack.append(pytesseract.image_to_string(cropped, lang='eng', config='--psm 4'))
        
        # textStack.append(pytesseract.image_to_string(cropped))
        # cv2.imshow('Cropped' + str(index), cropped)
        index = index + 1
        
    # write original image with added contours to disk
    while len(textStack) > 0:
        print(textStack.pop())
    
    cv2.imshow('Dilated', dilated)
    cv2.imshow('Text detection', img)
    

In [12]:
import cv2
import PIL.ImageGrab as ImageGrab
import numpy as np
import pytesseract

yCoord = 500
xCoord = 500
sizeY = 600
sizeX = 600

partOfFrameToShow = np.zeros((1,1))

while True:
    # img = ImageGrab.grab()
    img = cv2.imread('snapshot.png')
    img_np = np.array(img)
    frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)

    # partOfFrameToShow = frame[yCoord:yCoord+sizeY, xCoord:xCoord+sizeX]
    
    process_frame(frame)
    
    if cv2.waitKey(0) == 27:
        break   

cv2.destroyAllWindows()

import cv2
import PIL.ImageGrab as ImageGrab
import numpy as np
import pytesseract
yCoord = 500
xcoord = 580
sizeY = 600
sizeX = 600
partOfFframeToShow = np.zeros((1,1))
while True:
img = ImageGrab.grab()
# img = cv2. imread( ‘Snapshot. jpg’)
img_np = np.array(img)
frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
partOfFrameToShow = frame[yCoord:yCoord+sizeY, xCoord:xCoord+sizex]
captch_ex(partofFrameToShow)
if cv2.waitkey(5e)
break
cv2.destroyAllwindows()
Snapshot created.

